## Strategy Consider only Skewness

In [70]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("results.csv")

df

,Date,^IRX,^SPX,up_prob,down_prob,skewness,kurtosis,mean,variance,median,p10,p90
0,2011-01-07,0.135,1271.500000,0.3169,0.2072,1.0858,-0.3065,1270.7929,41522.7438,1318,1018,1529
1,2011-01-10,0.135,1269.750000,0.3346,0.2386,1.6923,1.4680,1374.2099,122425.1209,1292,999,1550
2,2011-01-11,0.145,1274.479980,0.3283,0.2448,1.6224,1.2034,1273.0806,53592.7590,1312,969,1539
3,2011-01-12,0.145,1285.959961,0.2935,0.2107,1.7602,1.7079,1409.5604,123446.8960,1301,1054,1515
4,2011-01-13,0.140,1283.760010,0.3139,0.2282,1.6123,1.1800,1402.6875,126110.2812,1300,1032,1541
...,...,...,...,...,...,...,...,...,...,...,...,...
2527,2021-01-25,0.070,3855.360107,0.2784,0.1905,1.6098,1.2175,3853.3838,370479.6495,4001,3106,4451
2528,2021-01-26,0.068,3849.620117,0.2814,0.1883,1.4036,0.5250,3847.3655,311757.0274,3983,3158,4511
2529,2021-01-27,0.060,3750.770020,0.3490,0.1511,1.5482,1.0250,3883.0176,249330.5602,3965,3178,4393
2530,2021-01-28,0.055,3787.379883,0.3123,0.1866,1.5422,0.9786,3785.5800,416746.6931,3958,3062,4445


In [71]:
# Define the sizes for train, validation, and test sets
train_size = int(len(df) * 0.333)
validation_size = int(len(df) * 0.333)
test_size = len(df) - train_size - validation_size

# Split the data without shuffling
train_set = df[:train_size]
validation_set = df[train_size:train_size + validation_size]
test_set = df[train_size + validation_size:]

In [72]:
from tqdm import tqdm

def trading_strategy(df, skewness_buy, skewness_sell):
    position = 0
    cumulative_returns = 0
    buy_price = 0
    signals = []
    for i in range(len(df)):
        signal = 'hold'

        # Check for buy signal
        if df['skewness'].iloc[i-1] > skewness_buy and position == 0:
            signal = 'buy'
            position = 1
            buy_price = df['^SPX'].iloc[i]

        # Check for sell signal
        elif df['skewness'].iloc[i-1] < skewness_sell and position == 1:
            signal = 'sell'
            position = 0
            sell_price = df['^SPX'].iloc[i]
            trade_return = (sell_price - buy_price) / buy_price
            cumulative_returns += trade_return

        signals.append(signal)

    return cumulative_returns, signals

def train_threshold(df, lr, std1, std2):
    best_return = -np.inf
    best_params = {}

    skewness_mean = df['skewness'].mean()
    skewness_std = df['skewness'].std()

    skewness_min = skewness_mean - std1 * skewness_std
    skewness_max = skewness_mean + std1 * skewness_std

    progress_bar = tqdm(total=(len(np.arange(skewness_min, skewness_max, lr)) ** 2))

    for skewness_buy in np.arange(skewness_min, skewness_max, lr):
        for skewness_sell in np.arange(skewness_min, skewness_max, lr):
            total_return, signals = trading_strategy(df, skewness_buy, skewness_sell)
            if total_return > best_return:
                best_return = total_return
                best_params = {
                    'skewness_buy': skewness_buy,
                    'skewness_sell': skewness_sell,
                    'skewness_buy_range': (skewness_buy - std2 * skewness_std, skewness_buy + std2 * skewness_std),
                    'skewness_sell_range': (skewness_sell - std2 * skewness_std, skewness_sell + std2 * skewness_std)
                }
            progress_bar.update(1)
    progress_bar.close()
    return best_return, best_params


In [73]:
def empirical_risk_minimization(shifted_signals, next_day_returns):
    indicator = [1 if signal == 'buy' else -1 if signal == 'sell' else 0 for signal in shifted_signals]
    empirical_risk = -np.sum(next_day_returns * indicator)
    return empirical_risk

def calculate_expected_profit(df, spx_return, forecasted_cum_returns, y2_forecasted):

    mu_1 = df[spx_return].mean()
    mu_2 = df[forecasted_cum_returns].mean()
    sigma_1 = df[spx_return].std()
    sigma_2 = df[forecasted_cum_returns].std()
    sigma_12 = df[spx_return].cov(df[forecasted_cum_returns])
    rho = df[spx_return].corr(df[forecasted_cum_returns])

    z_score_forecasted = mu_2 / sigma_2
    phi = norm.cdf(z_score_forecasted)
    part1 = mu_1 * phi
    part2 = (sigma_12 / (sigma_2 * np.sqrt(2 * np.pi))) * np.exp(-0.5 * z_score_forecasted**2)

    expected_profit = part1 + part2

    return np.where(y2_forecasted > -mu_2 / (sigma_2), expected_profit, 0)

def optimize_thresholds(df, spx_return, forecasted_cum_returns, y2_forecasted, trading_range, lr, patience):

    y2_forecasted_aligned = y2_forecasted.shift(-1).dropna()  # Align forecasts ????
    # y2_forecasted_aligned = y2_forecasted

    best_score = float('-inf')
    best_expected_profits = 0
    best_empirical_risk = 0
    best_thresholds = {
        'skewness_buy': None,
        'skewness_sell': None
    }
    score_function = lambda profit, risk: profit - risk

    skewness_buy_range = np.arange(*trading_range['skewness_buy_range'], lr)
    skewness_sell_range = np.arange(*trading_range['skewness_sell_range'], lr)

    no_improvement_count = 0  # Counter for early stopping

    # Grid search within the adjusted ranges
    for skewness_buy in skewness_buy_range:
            for skewness_sell in skewness_sell_range:
                    returns, signals = trading_strategy(df, skewness_buy, skewness_sell)
                    empirical_risk = empirical_risk_minimization(signals[:-1], df['spx_return'].iloc[1:].reset_index(drop=True))
                    expected_profits = calculate_expected_profit(df, spx_return, forecasted_cum_returns, y2_forecasted)[0]
                    score = score_function(expected_profits, empirical_risk)

                    if score > best_score:
                        best_score = score
                        best_expected_profits = expected_profits
                        best_empirical_risk = empirical_risk
                        best_thresholds = {
                            'skewness_buy': skewness_buy,
                            'skewness_sell': skewness_sell
                        }
                        no_improvement_count = 0  # Reset counter if there's an improvement
                    else:
                        no_improvement_count += 1  # Increment counter if no improvement

                    # If no improvement for a number of iterations, stop the search
                    if no_improvement_count >= patience:
                        print("Early stopping triggered.")
                        return best_thresholds, best_score, best_expected_profits, best_empirical_risk

    return best_thresholds, best_score, best_expected_profits, best_empirical_risk

def process_in_batches(df, batch_size, spx_return, forecasted_cum_returns, y2_forecasted, trading_range, lr, patience):
    results = []

    # Using tqdm to create a progress bar over the range
    for start in tqdm(range(0, len(df), batch_size), desc='Processing batches'):
        end = min(start + batch_size, len(df))
        df_batch = df.iloc[start:end]
        y2_batch = y2_forecasted[start:end].shift(-1).dropna()  # Align y2_forecasted for each batch

        best_thresholds, best_score, best_expected_profits, best_empirical_risk = optimize_thresholds(df_batch, spx_return, forecasted_cum_returns, y2_batch, trading_range, lr, patience)

        results.append({
            'Batch Start': start,
            'Batch End': end - 1,
            'Skewness Buy Threshold': best_thresholds['skewness_buy'],
            'Skewness Sell Threshold': best_thresholds['skewness_sell'],
            'Best Score': best_score,
            'Min Risk': best_empirical_risk,
            'Max Expected Profit':best_expected_profits

        })

    return pd.DataFrame(results)

In [74]:
# Calculate actual return
train_set['spx_return'] = train_set['^SPX'].pct_change()
validation_set['spx_return'] = validation_set['^SPX'].pct_change()
test_set['spx_return'] = test_set['^SPX'].pct_change()

## lr = 0.5, 0.4, 0.3, 0.2, 0.1
## First locate best lr, while others constant

In [75]:
std1 = 2.0
std2 = 1.0
patience = 5
batch_size = 300

In [76]:
lr = 0.5

# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 91.96it/s]


In [77]:
results_df

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,1.282164,1.282164,0.057693,-0.055504,0.002188
1,300,599,0.782164,1.282164,0.102055,-0.102055,0.000000
2,600,842,1.282164,1.282164,-0.035315,0.036732,0.001417


In [78]:
lr = 0.4


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_1 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 83.93it/s]


In [79]:
results_df_1

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,0.282164,1.482164,0.109276,-0.106792,0.002484
1,300,599,0.682164,1.482164,-0.012747,0.015885,0.003138
2,600,842,0.282164,1.882164,0.071917,-0.070085,0.001832


In [80]:
lr = 0.3


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_2 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 53.00it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [81]:
results_df_2

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,0.282164,1.182164,0.037175,-0.036270,0.000905
1,300,599,0.282164,1.182164,0.075305,-0.074461,0.000844
2,600,842,0.282164,0.582164,-0.004278,0.004956,0.000678


In [82]:
lr = 0.2

# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_3 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 40.91it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [83]:
results_df_3

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,0.482164,1.082164,0.076310,-0.075214,0.001096
1,300,599,0.682164,1.282164,0.180896,-0.179916,0.000980
2,600,842,0.482164,0.682164,0.004469,-0.003813,0.000656


In [84]:
(0.076310 + 0.180896	+ 0.004469)/3

0.087225

In [85]:
lr = 0.1


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_4 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 46.59it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [86]:
results_df_4

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,0.482164,1.082164,0.076310,-0.075214,0.001096
1,300,599,0.482164,0.982164,0.135977,-0.134997,0.000980
2,600,842,0.482164,0.682164,0.004469,-0.003813,0.000656


In [87]:
(0.076310	 + 0.135977 + 0.004469)/3

0.072252

## Choose lr = 0.2 according to the best Best score in Batch Processing

## std = (1.0, 1.0) (2.0, 2.0) (2.0, 1.0) (3.0, 3.0) (3.0, 2.0) (3.0, 1.0)
## Then locate best std1 and std2, while others constant and lr = 0.2

In [88]:
lr = 0.2
patience = 5
batch_size = 300

In [89]:
std1 = 1.0
std2 = 1.0


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

100%|██████████| 16/16 [00:00<00:00, 162.76it/s]


Early stopping triggered.

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 48.00it/s]


Early stopping triggered.
Early stopping triggered.


In [90]:
results_df

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,0.825115,1.225115,0.040065,-0.038961,0.001105
1,300,599,0.825115,1.025115,0.106309,-0.105298,0.001011
2,600,842,0.825115,0.625115,-0.004317,0.004956,0.000639


In [91]:
std1 = 2.0
std2 = 2.0


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_1 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 39.25it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [92]:
results_df_1

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,0.139213,1.539213,0.075833,-0.074737,0.001096
1,300,599,0.139213,1.139213,0.120868,-0.119888,0.000980
2,600,842,0.139213,0.339213,-0.004299,0.004956,0.000656


In [93]:
std1 = 2.0
std2 = 1.0


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_2 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 38.61it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [94]:
results_df_2

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,0.482164,1.082164,0.076310,-0.075214,0.001096
1,300,599,0.682164,1.282164,0.180896,-0.179916,0.000980
2,600,842,0.482164,0.682164,0.004469,-0.003813,0.000656


In [95]:
std1 = 3.0
std2 = 3.0

# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_3 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 42.55it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [96]:
results_df_3

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.74669,1.65331,0.267614,-0.265046,0.002568
1,300,599,-0.74669,1.05331,0.183557,-0.180248,0.003309
2,600,842,-0.74669,2.05331,0.099701,-0.097857,0.001845


In [97]:
std1 = 3.0
std2 = 2.0


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_4 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 46.70it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [98]:
results_df_4

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.403739,1.796261,0.067703,-0.065135,0.002568
1,300,599,-0.403739,1.196261,0.077770,-0.074461,0.003309
2,600,842,-0.403739,1.996261,0.092259,-0.090414,0.001845


In [99]:
std1 = 3.0
std2 = 1.0


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_5 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 48.18it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [100]:
results_df_5

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.060787,1.539213,0.077305,-0.074737,0.002568
1,300,599,-0.060787,2.139213,0.068030,-0.064721,0.003309
2,600,842,-0.060787,2.139213,0.078141,-0.076296,0.001845


## Choose std = (3.0, 3.0) according to the best Best score in Batch Processing


## Batch_size = 100, 200, 300, 400, 500
## Then locate best Batch_size, while others constant and lr = 0.2, std = (3.0, 3.0)


In [101]:
lr = 0.2
std1 = 3.0
std2 = 3.0
patience = 5

In [102]:
batch_size = 100


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 9/9 [00:00<00:00, 81.85it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.
Early stopping triggered.
Early stopping triggered.
Early stopping triggered.
Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [103]:
results_df

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,99,-0.74669,1.25331,0.046830,-0.044810,0.002020
1,100,199,-0.74669,1.65331,0.089083,-0.085713,0.003371
2,200,299,-0.74669,1.65331,0.170146,-0.170146,0.000000
3,300,399,-0.74669,1.05331,0.149357,-0.149357,0.000000
4,400,499,-0.74669,1.25331,0.102132,-0.098718,0.003414
5,500,599,-0.74669,1.85331,0.069495,-0.066898,0.002598
6,600,699,-0.74669,2.25331,0.065237,-0.063184,0.002053
7,700,799,-0.74669,2.05331,0.022789,-0.021056,0.001733
8,800,842,-0.74669,1.45331,0.025579,-0.023924,0.001655


In [104]:
batch_size = 200


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_1 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 5/5 [00:00<00:00, 58.46it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [105]:
results_df_1

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,199,-0.74669,1.65331,0.158153,-0.155403,0.002750
1,200,399,-0.74669,1.65331,0.194871,-0.194871,0.000000
2,400,599,-0.74669,1.25331,0.096908,-0.093874,0.003034
3,600,799,-0.74669,2.05331,0.084872,-0.082983,0.001890
4,800,842,-0.74669,1.45331,0.025579,-0.023924,0.001655


In [106]:
batch_size = 300


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_2 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 43.06it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [107]:
results_df_2

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.74669,1.65331,0.267614,-0.265046,0.002568
1,300,599,-0.74669,1.05331,0.183557,-0.180248,0.003309
2,600,842,-0.74669,2.05331,0.099701,-0.097857,0.001845


In [108]:
batch_size = 400


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_3 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 34.99it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [109]:
results_df_3

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,399,-0.74669,1.65331,0.292713,-0.289771,0.002942
1,400,799,-0.74669,2.05331,0.127143,-0.124589,0.002554
2,800,842,-0.74669,1.45331,0.025579,-0.023924,0.001655


In [110]:
batch_size = 500


# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_4 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 2/2 [00:00<00:00, 31.30it/s]

Early stopping triggered.
Early stopping triggered.


In [111]:
results_df_4

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,499,-0.74669,1.05331,0.224593,-0.221556,0.003037
1,500,842,-0.74669,1.85331,0.139054,-0.136953,0.002101


## Choose batch_size = 300 according to the best Best score in Batch Processing
## patience = 1, 2, 3, 4, 5
## Then locate best patience, while lr = 0.2, std = (3.0., 3.0), batch_size = 300

In [112]:
lr = 0.2
std1 = 3.0
std2 = 3.0
batch_size = 300

In [113]:
patience = 1

# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 144.96it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [114]:
results_df

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.74669,1.05331,0.054516,-0.051949,0.002568
1,300,599,-0.74669,1.05331,0.183557,-0.180248,0.003309
2,600,842,-0.74669,0.85331,-0.018508,0.020353,0.001845


In [115]:
patience = 2

# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_1 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 63.97it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [116]:
results_df_1

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.74669,1.65331,0.267614,-0.265046,0.002568
1,300,599,-0.74669,1.05331,0.183557,-0.180248,0.003309
2,600,842,-0.74669,2.05331,0.099701,-0.097857,0.001845


In [117]:
patience = 3

# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_2 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 54.53it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [118]:
results_df_2

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.74669,1.65331,0.267614,-0.265046,0.002568
1,300,599,-0.74669,1.05331,0.183557,-0.180248,0.003309
2,600,842,-0.74669,2.05331,0.099701,-0.097857,0.001845


In [119]:
patience = 4

# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_3 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 46.38it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [120]:
results_df_3

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.74669,1.65331,0.267614,-0.265046,0.002568
1,300,599,-0.74669,1.05331,0.183557,-0.180248,0.003309
2,600,842,-0.74669,2.05331,0.099701,-0.097857,0.001845


In [121]:
patience = 5

# # Get thresholds trained by train_set
best_return, best_params = train_threshold(train_set, lr, std1, std2)

skewness_buy = best_params['skewness_buy']
skewness_sell = best_params['skewness_sell']


# # Calculate forecast return using validation set - forcaste cumulatiev return (Y2) by spx_return (Y1) using signals
returns, signals = trading_strategy(validation_set, skewness_buy, skewness_sell)
validation_set['forecasted_cum_returns'] = 0.0
position = 0
buy = None

for i in range(1, len(validation_set)):
    if signals[i-1] == 'buy' and position == 0:
        position = 1
        buy = i

    if signals[i-1] == 'sell' and position == 1:
        sell = i
        # Calculate cumulative return for the holding period
        validation_set['forecasted_cum_returns'].iloc[buy:sell+1] = validation_set['spx_return'].iloc[buy:sell+1].cumsum()
        position = 0

# After the loop, if the last position was not closed, calculate the cumulative return for the remaining period
if position == 1:
    validation_set['forecasted_cum_returns'].iloc[buy:] = validation_set['spx_return'].iloc[buy:].cumsum()

results_df_4 = process_in_batches(validation_set, batch_size, 'spx_return', 'forecasted_cum_returns', validation_set['forecasted_cum_returns'], best_params, lr, patience)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 38.79it/s]

Early stopping triggered.
Early stopping triggered.
Early stopping triggered.


In [122]:
results_df_4

,Batch Start,Batch End,Skewness Buy Threshold,Skewness Sell Threshold,Best Score,Min Risk,Max Expected Profit
0,0,299,-0.74669,1.65331,0.267614,-0.265046,0.002568
1,300,599,-0.74669,1.05331,0.183557,-0.180248,0.003309
2,600,842,-0.74669,2.05331,0.099701,-0.097857,0.001845


## Since the best score is not imporved after patience = 2, which has better best score than patience = 1, we just choose patience = 5

## In conclusion, we have best lr = 0.2, std1 = 3.0, std2 =  3.0, size = 300, and patience = 5